In [ ]:
%load_ext autoreload
%autoreload 2
from ipywidgets import widgets
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from utilities import load_mappings, extract_faults


In [ ]:
base_mapped_faults, base_file_sizes = load_mappings("example")
test_mapped_faults, test_file_sizes = load_mappings("example")


In [ ]:
# Whether to include minor page faults in analysis
page_fault_checkbox = widgets.Checkbox(
    description="Include minor page faults",
    disabled=False,
)

# The file to analyze
base_file_name_widget = widgets.Dropdown(
    options= list(set(m["file_name"] for m in base_mapped_faults)),
    description="Base File Name:",
    layout=widgets.Layout(padding="0px 0px 200px 0px", width="800px"),
    disabled=False,
)

test_file_name_widget = widgets.Dropdown(
    options= list(set(m["file_name"] for m in test_mapped_faults)),
    description="Test File Name:",
    layout=widgets.Layout(padding="0px 0px 200px 0px", width="800px"),
    disabled=False,
)


display(page_fault_checkbox)
display(base_file_name_widget)
display(test_file_name_widget)

# Param: If applicable, the name of file embedded within APK
# This is useful if native libraries are embedded within the APK instead of extracted out
apk_entry_name = None


In [ ]:
base_faults, _, _ = extract_faults(base_file_name_widget.value, apk_entry_name, base_file_sizes, base_mapped_faults, page_fault_checkbox.value)
test_faults, _, _ = extract_faults(test_file_name_widget.value, apk_entry_name, test_file_sizes, test_mapped_faults, page_fault_checkbox.value)


In [ ]:
def compute_summary_statistics(df):
  """Computes summary statistics for a given dataframe."""
  num_page_faults = df['offset'].count()
  num_major_page_faults = df[df['is_major'] == True]['offset'].count()

  if not df.empty:
    time_diff = df['ts'].max() - df['ts'].min()
  else:
    time_diff = None

  print(f"Elapsed Time: {time_diff}, {num_page_faults} Page Faults , {num_major_page_faults} Major Page Faults")

print("Base:")
compute_summary_statistics(base_faults)
print("Test:")
compute_summary_statistics(test_faults)

In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, subplot_titles=("Base Offset by Index", "Test Offset by Index"))

fig.add_trace(go.Scatter(
    x=base_faults.index,
    y=base_faults['offset'],
    mode='markers',
    name='Base',
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=test_faults.index,
    y=test_faults['offset'],
    mode='markers',
    name='Test',
), row=2, col=1)

fig.update_layout(
    title="Offsets (Base vs. Test)",
    height=600,
    width=1400,
    showlegend=True
).show()


In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, subplot_titles=("Base Offset by Time", "Test Offset by Time"))

fig.add_trace(go.Scatter(
    x=base_faults['ts'],
    y=base_faults['offset'],
    mode='markers',
    name='Base',
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=test_faults['ts'],
    y=test_faults['offset'],
    mode='markers',
    name='Test',
), row=2, col=1)

fig.update_layout(
    title="Offsets (Base vs. Test)",
    height=600,
    width=1400,
    showlegend=True
).show()

In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, subplot_titles=("Base Cumulative Page Faults Over Time", "Test Cumulative Page Faults Over Time"))

fig.add_trace(go.Scatter(
    x=base_faults['ts'],
    y=base_faults.index,
    mode='lines',
    name='Base',
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=test_faults['ts'],
    y=test_faults.index,
    mode='lines',
    name='Test',
), row=2, col=1)

fig.update_layout(
    height=600,
    width=1400,
    showlegend=True
).show()


In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, subplot_titles=("Base Offset by Index", "Test Offset by Index"))

fig.add_trace(go.Scatter(
    x=base_faults.index,
    y=base_faults['offset_diff'],
    mode='lines',
    name='Base',
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=test_faults.index,
    y=test_faults['offset_diff'],
    mode='lines',
    name='Test',
), row=2, col=1)

fig.update_layout(
    height=700,
    width=1500,
    showlegend=True
).show()
